## Import all the required libraries

 The libraries need are for **Reading files, Data Manipulation, Data Transformation, Data Preprocessing, Machine Learning Models, Evaluation of Performance Metric and Data Validation**.

In [3]:
# Read NetCDF file
import xarray as xr

# Data Manipulation
import pandas as pd
import numpy as np

# Data Transformation (Coordinates)
import pyproj

# Data Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Machine Learning Models
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor as rf

# Performance metric
from sklearn.metrics import mean_squared_error

#Model Validation
from sklearn.model_selection import cross_validate


### Import the Bathymetry dataset

In [5]:
bathyData_raw = xr.open_dataset('../Data/Raw/gebco_2025_n5.6114_s4.4256_w2.541_e3.6704.nc')

# View the contents of the dataset
bathyData_raw


<xarray.Dataset>
Dimensions:    (lat: 285, lon: 271)
Coordinates:
  * lat        (lat) float64 4.427 4.431 4.435 4.44 ... 5.598 5.602 5.606 5.61
  * lon        (lon) float64 2.544 2.548 2.552 2.556 ... 3.656 3.66 3.665 3.669
Data variables:
    elevation  (lat, lon) int16 ...
Attributes: (12/36)
    title:                           The GEBCO_2025 Grid - a continuous terra...
    summary:                         The GEBCO_2025 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/37c52e96-24ea-67ce-e063-708...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/37c52e96-24ea-67ce-e063-708...
    references:                      DOI: 10.5285/37c52e96-24ea-67ce-e063-708...
    node_offset:                     1.0

### Extract the Position (latitude and longitude) and Depth data from the xarray

The xarray has a lot of information within it. For the purpose of this project, only the numerical values within the **lat**, **lon** and **elevation** varaibles would be extracted.

In [8]:
latitude  = bathyData_raw["lat"].values
longitude = bathyData_raw["lon"].values
depth = bathyData_raw["elevation"].values


### Check the dimensions of the array

In [10]:
print(
    "Dimension of the arrays \n"
    f"Latitude: {latitude.ndim}\n"
    f"Longitude: {longitude.ndim}\n"
    f"Depth: {depth.ndim}"
)


Dimension of the arrays 
Latitude: 1
Longitude: 1
Depth: 2


### Harmonize the data to the same dimension to form a grid

In [12]:
X,Y = np.meshgrid(longitude, latitude)

harmonized_data = pd.DataFrame({
    "Lat": X.ravel(),
    "Lon": Y.ravel(),
    "Depth": depth.ravel()
})

harmonized_data


,Lat,Lon,Depth
0,2.543750,4.427083,-3862
1,2.547917,4.427083,-3859
2,2.552083,4.427083,-3858
3,2.556250,4.427083,-3857
4,2.560417,4.427083,-3854
...,...,...,...
77230,3.652083,5.610417,-2346
77231,3.656250,5.610417,-2343
77232,3.660417,5.610417,-2338
77233,3.664583,5.610417,-2340


In [13]:
harmonized_data.describe()


,Lat,Lon,Depth
count,77235.000000,77235.000000,77235.000000
mean,3.106250,5.018750,-3162.308267
std,0.325962,0.342802,367.236616
min,2.543750,4.427083,-3899.000000
25%,2.822917,4.722917,-3473.000000
50%,3.106250,5.018750,-3176.000000
75%,3.389583,5.314583,-2860.000000
max,3.668750,5.610417,-2282.000000


### Save the extracted Bathy Data

In [15]:
harmonized_data.to_csv("../Data/Processed/01_Extracted Raw Bathymetry Data (Lat & Lon).csv", index = False)


### Convert the coordinates (lat and lon) to UTM to be on uniform scale

1° latitude is not equal to 1° longitude

In [18]:
project = pyproj.Proj(proj = "utm", zone = "31", ellipsoid = "WGS84")
raw_bathy = harmonized_data.copy()
raw_bathy["Easting"], raw_bathy["Northing"] = project(raw_bathy["Lon"], raw_bathy["Lat"])


### Preview the new data (UTM coordinates)

In [20]:
raw_bathy.head()


,Lat,Lon,Depth,Easting,Northing
0,2.543750,4.427083,-3862,658659.665683,281250.367360
1,2.547917,4.427083,-3859,658659.155888,281711.061633
2,2.552083,4.427083,-3858,658658.645259,282171.755933
3,2.556250,4.427083,-3857,658658.133797,282632.450262
4,2.560417,4.427083,-3854,658657.621500,283093.144619


### Exploratory Data Analysis

In [22]:
bathyData_df = raw_bathy.copy()
bathyData_df


,Lat,Lon,Depth,Easting,Northing
0,2.543750,4.427083,-3862,658659.665683,281250.367360
1,2.547917,4.427083,-3859,658659.155888,281711.061633
2,2.552083,4.427083,-3858,658658.645259,282171.755933
3,2.556250,4.427083,-3857,658658.133797,282632.450262
4,2.560417,4.427083,-3854,658657.621500,283093.144619
...,...,...,...,...,...
77230,3.652083,5.610417,-2346,789988.268826,404091.432067
77231,3.656250,5.610417,-2343,789986.929143,404552.470439
77232,3.660417,5.610417,-2338,789985.587933,405013.508845
77233,3.664583,5.610417,-2340,789984.245199,405474.547286


### Filter, rearrange the columns and get more information about the dataframe

In [24]:
bathyData_df.drop(columns = ["Lat", "Lon"], inplace = True)

new_pattern = ["Northing", "Easting", "Depth"]
bathyData_df = bathyData_df[new_pattern]

print(bathyData_df.head())

print("\n")                                           #Introduce an empty line

print("bathyData_df Shape: ", bathyData_df.shape)     # Number of observations and features

print("\n")                                           #Introduce an empty line

bathyData_df.info()                                   # Descriptive information about the dataset


        Northing        Easting  Depth
0  281250.367360  658659.665683  -3862
1  281711.061633  658659.155888  -3859
2  282171.755933  658658.645259  -3858
3  282632.450262  658658.133797  -3857
4  283093.144619  658657.621500  -3854


bathyData_df Shape:  (77235, 3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77235 entries, 0 to 77234
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Northing  77235 non-null  float64
 1   Easting   77235 non-null  float64
 2   Depth     77235 non-null  int16  
dtypes: float64(2), int16(1)
memory usage: 1.3 MB


### Save the transformed Bathy Data

In [26]:
bathyData_df.to_csv("../Data/Processed/02_Transformed Raw Bathymetry Data (UTM).csv", index = False)


### Summary of the statistics for the data

In [28]:
bathyData_df.describe()


,Northing,Easting,Depth
count,77235.000000,77235.000000,77235.000000
mean,343557.863898,724355.368096,-3162.308267
std,36052.805085,38113.126318,367.236616
min,281250.367360,658491.749220,-3899.000000
25%,312276.178449,691413.586159,-3473.000000
50%,343551.572859,724354.955788,-3176.000000
75%,374824.974316,757314.391738,-2860.000000
max,405935.585760,790290.429339,-2282.000000


### Check for missing data in the DataFrame

In [30]:
bathyData_df.isnull().sum()


Northing    0
Easting     0
Depth       0
dtype: int64

### Simulate missing data (NaN) for the models

In [32]:
np.random.seed(42)
mask = np.random.rand(len(harmonized_data)) < 0.1
print("\n")


***

## KNNImputer

### Add the mssing data (NaN) into the DataFrame

In [36]:
true_data = bathyData_df.copy()
missingData_df = true_data.copy()

missingData_df.loc[mask, "Depth"] = np.nan


### View missingData_df to ensure the presence of missing data points (NaN)

In [38]:
missingData_df.isnull().sum()


Northing       0
Easting        0
Depth       7725
dtype: int64

### Scale the numerical data

In [40]:
# Only the observations with the complete data (Lat, lon and depth) were scaled to avoid 
# compromise of our missing data and overfitting our model

scaler_coords = MinMaxScaler()
scaler_depth = MinMaxScaler()

mask_depth = missingData_df["Depth"].isna()

# Fit the scaler on complete data (section of the data without NaN values)
scaler_coords.fit(missingData_df.loc[~mask_depth, ["Northing", "Easting"]])
scaler_depth.fit(missingData_df.loc[~mask_depth, ["Depth"]])

#Transform the entire dataset 
scaled_coords = scaler_coords.transform(missingData_df[["Northing", "Easting"]])
scaled_depth = scaler_depth.transform(missingData_df[["Depth"]])
                                     
# Combine the 2 arrays (scaled data) into one array
scaler_combined = np.hstack([scaled_coords, scaled_depth])    


### Save the masked & scaled bathy data

In [42]:
scaler_combined_toDF = pd.DataFrame(scaler_combined, columns = ["Northing", "Easting", "Depth"])

scaler_combined_toDF.to_csv("../Data/Processed/03_Masked & Scaled data for KNNImputer.csv", index = False)


### Populate the missing depth with KNNImputer (K-Nearest Neighbors)

In [44]:
imputer = KNNImputer(n_neighbors = 5, weights = "distance") 
Imputed_data = imputer.fit_transform(scaler_combined)


### Extract the newly predicted depth from Imputed_data and convert it to its original scale

In [46]:
imputedDepth = Imputed_data[:, 2]
imputedDepth.ndim            #Dimension of the array


1

Transform imputedDepth from a 1D array to a 2D column vector

In [48]:
# N.B: The MinMax scaler() only accepts 2D inputs.

imputedDepth = imputedDepth.reshape(-1, 1)
imputedDepth.ndim             #Cross-check dimension after the transformation


2

### Reverse the scaled data to the actual units

In [50]:
Depth_originalScale = scaler_depth.inverse_transform(imputedDepth)


In [51]:
Bathy_KNNImputer = missingData_df.copy()
Bathy_KNNImputer["Depth"] = Depth_originalScale
Bathy_KNNImputer.head()


,Northing,Easting,Depth
0,281250.367360,658659.665683,-3862.0
1,281711.061633,658659.155888,-3859.0
2,282171.755933,658658.645259,-3858.0
3,282632.450262,658658.133797,-3857.0
4,283093.144619,658657.621500,-3854.0


### Check for any missing depth data

In [53]:
Bathy_KNNImputer.isnull().sum()


Northing    0
Easting     0
Depth       0
dtype: int64

### Convert the coordinates back to Latitude and Longitude

In [55]:
Bathy_KNNImputer["Longitude"], Bathy_KNNImputer["Latitude"] = project(Bathy_KNNImputer["Easting"], Bathy_KNNImputer["Northing"], inverse = True)
Bathy_KNNImputer.drop(columns = ["Northing", "Easting"], inplace = False)

new_order = ["Latitude", "Longitude", "Depth"]
Bathy_KNNImputer = Bathy_KNNImputer[new_order]
Bathy_KNNImputer.head()


,Latitude,Longitude,Depth
0,2.543750,4.427083,-3862.0
1,2.547917,4.427083,-3859.0
2,2.552083,4.427083,-3858.0
3,2.556250,4.427083,-3857.0
4,2.560417,4.427083,-3854.0


### Compute the performance metrics of the KNNImputer

For this project, we would evalute the Root Mean Squared Error (RMSE) of KNNImputer.

In [58]:
true_depth = true_data.loc[mask_depth, "Depth"].values
predicted_depth = Bathy_KNNImputer.loc[mask_depth, "Depth"].values


RMSE = np.sqrt(mean_squared_error(true_depth, predicted_depth))
print(f"The Root Mean Squared Error (RMSE) of KNN is: {RMSE:.2f}m")

print("\n")


The Root Mean Squared Error (RMSE) of KNN is: 3.29m




---

## RANDOM FOREST MODEL

### Add the missing data (NaN) into the DataFrame

In [62]:
actual_df = raw_bathy.copy()
actual_df.drop(columns = ["Northing", "Easting"], inplace = True)
df_dataGaps = actual_df.copy()

df_dataGaps.loc[mask, "Depth"] = np.nan


### View missingData_df to ensure the introduction of missing data points (NaN)

In [64]:
df_dataGaps.isna().sum()


Lat         0
Lon         0
Depth    7725
dtype: int64

### Save the masked bathy data

In [66]:
df_dataGaps.to_csv("../Data/Processed/04_Masked data for Random Forest Model.csv", index = False)


### Define the train data (X) and target data (y)

In [68]:
X = df_dataGaps.loc[~mask,["Lat", "Lon"]]
y = df_dataGaps.loc[~mask, "Depth"]
predicted_X = df_dataGaps.loc[mask, ["Lat", "Lon"]]


### Instantiate the model

In [70]:
model = rf(n_estimators=100, random_state=42)
model.fit(X, y)


RandomForestRegressor(random_state=42)

### Predict the missing depth (NaN) values

In [72]:
predicted_y = model.predict(predicted_X)


### Input the predicted_y (data) into the dataframe

In [74]:
df_dataGaps.loc[mask, "Depth"] = predicted_y


### Inspect the DataFrame to ensure there's no any missing depth data

In [76]:
Bathy_RF = df_dataGaps.copy()
Bathy_RF["Depth"] = np.round(Bathy_RF["Depth"], decimals = 2)

Bathy_RF.isna().sum()


Lat      0
Lon      0
Depth    0
dtype: int64

### Compute the Performance metrics of the RF model

For this project, we would evalute the Root Mean Squared Error (RMSE) of the RF model.

In [79]:
actual_depth = actual_df.loc[mask, "Depth"].values
predicted_depth = Bathy_RF.loc[mask, "Depth"].values


RMSE = np.sqrt(mean_squared_error(actual_depth , predicted_depth))

print(f"The Root Mean Squared Error (RMSE) of the RandomForest model is: {RMSE:.2f}m")


The Root Mean Squared Error (RMSE) of the RandomForest model is: 3.28m


###  Compare the Actual Depth with Predicted values from the 2 models (KNN & RF)

In [81]:
Depth_Comparison = pd.DataFrame({
    
    "Lat": harmonized_data.loc[mask, "Lat"],
     "Lon": harmonized_data.loc[mask, "Lon"],
     "Depth": harmonized_data.loc[mask, "Depth"],
    "Depth (KNNImputer)": Bathy_KNNImputer.loc[mask, "Depth"].round(2),
    "Depth (RF)" : Bathy_RF.loc[mask, "Depth"].round(2)
    
    })
Depth_Comparison


,Lat,Lon,Depth,Depth (KNNImputer),Depth (RF)
6,2.568750,4.427083,-3835,-3830.56,-3826.01
10,2.585417,4.427083,-3829,-3829.77,-3829.60
29,2.664583,4.427083,-3789,-3792.40,-3799.78
32,2.677083,4.427083,-3806,-3811.08,-3807.54
37,2.697917,4.427083,-3795,-3797.77,-3799.08
...,...,...,...,...,...
77156,3.343750,5.610417,-2697,-2704.12,-2702.25
77174,3.418750,5.610417,-2611,-2615.46,-2615.22
77205,3.547917,5.610417,-2472,-2471.77,-2473.29
77211,3.572917,5.610417,-2436,-2437.84,-2437.57


### Save the Depth_Comparison data data frame

In [83]:
Depth_Comparison.to_csv("../Results/Predicted Bathymetry Data.csv", index = False)


### Perform cross-validation on the Random Forest model

In [85]:
cv_results = cross_validate(model, X, y, cv = 5, return_train_score = True)
scores = cv_results["test_score"]

print(f"The mean cross-validation accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")


The mean cross-validation accuracy is: 0.922 ± 0.069


In [135]:
print(f"The train_score is: {cv_results["train_score"].mean():.3f}")

The train_score is: 1.000


### Conclusion

Upon comparison of the training and test scores, it is concluded that the **Random Forest model generalizes well on unseen data**.